**bert Training**

Importing useful libraries at first

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import pandas as pd
import numpy as np

import sklearn
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
%matplotlib inline
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 200
SEED = 100

import warnings
warnings.filterwarnings("ignore")

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

import torch
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

np.random.seed(SEED)
num_labels =  15
pos_weight = 10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

In [ ]:
# to check if PyTorch is capable of using GPU, run the following code.

import torch
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

In [ ]:
# Importing the database 

with open('/content/drive/MyDrive/thesis/feature_eng_dataframe.pkl', 'rb') as f:
    df_train,df_val,df_test,tag_cols = pickle.load(f)

In [ ]:
df_train.head(5)

,text,labels
6867,place need compare 2 nullable value see think something framework support find anything instead follow public static bool isdifferentto bool x bool return xhasvalue yhasvalue true xhasvalue amp xv...,"(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
18494,codehtml file contain follow code ajax type post datatype jsonp url path success function msg var e documentcreateelement div eid ads documentbodyappendchild e ads html msg open codehtml file brow...,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0)"
29363,check whether bootstrapjs load page file bootstrapjs may compile another big js file,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)"
36595,try run wordcount job hadoopbut always get class find exceptioni post class write command use run job import javaioioexception import javautil import orgapachehadoopfspath import orgapachehadoopco...,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0)"
24197,logcat see lot dsr send dtr android tether phone wirelessly mac internet dev mode also mean see something bluetooth though dsr dtr,"(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"


### Modelling - SimpleTransformers


In [ ]:
!pip install simpletransformers
!pip install h5py
!pip install wandb --upgrade  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel
from simpletransformers.classification import classification_model
import logging
import wandb

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

Defining ModelArgs for bert model

In [ ]:
# The ModelArgs (train_args) dataclass contains all the global options set to their default values

sweep_config = {
    "name": "stacksample-hyperparameter-bert",
    "method": "bayes",  
    "metric": {"name": "eval_loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 3e-5, "max": 7e-5},
        "train_batch_size": {"values": [32, 64, 128]},
        "gradient_accumulation_steps": {"values": [8, 16]},
        "max_seq_length": {"values": [128, 256]},
    }
}

model_type = 'bert'
model_name = 'bert-base-uncased'

sweep_id = wandb.sweep(sweep_config, project="stacksample-hyperparameter-bert")

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,

    # paths
    "output_dir": f"/content/drive/MyDrive/thesis/output/bert_tuning/final",
    "best_model_dir": f"/content/drive/MyDrive/thesis/output/bert_tuning/best_model",

    # size
    #"train_batch_size": 64, # 
    #"max_seq_length": 128, # use small value to avoid OOM
    "num_train_epochs": 10,

    # rates
    # "weight_decay": 0,
    #"learning_rate": 3e-5,
    # "adam_epsilon": 1e-8,

    # evaluation
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "evaluate_during_training_verbose": False,
    "sliding_window": False,
    #"eval_batch_size": 64,
    #"gradient_accumulation_steps": 16,
    "wandb_project" : "stacksample-hyperparameter-bert",
    "use_early_stopping":True,
    "early_stopping_patience": 5,
    "early_stopping_delta": 0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,

}

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: oh1frsnh
Sweep URL: https://wandb.ai/ranvir_rana/stacksample-hyperparameter-bert/sweeps/oh1frsnh


Initializing a ClassificationModel for bert model

In [ ]:
def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = MultiLabelClassificationModel(model_type, model_name,args=train_args,num_labels=num_labels, use_cuda=True,sweep_config=wandb.config)

    # Train the model
    model.train_model(df_train, eval_df=df_val)

    # Evaluate the model
    model.eval_model(df_val)

    # Sync wandb
    wandb.join()

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 9sfbyhgq with config:
wandb: 	gradient_accumulation_steps: 16
wandb: 	learning_rate: 5.442661686434627e-05
wandb: 	max_seq_length: 256
wandb: 	train_batch_size: 32
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ranvir_rana. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not 

  0%|          | 0/26700 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/835 [00:00<?, ?it/s]

...

Run qgex88e3 errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 14.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run qgex88e3 errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 14.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: xrfxfl3o with config:
wandb: 	gradient_accumulation_steps: 8
wandb: 	learning_rate: 5.444236434248727e-05
wandb: 	max_seq_length: 128
wandb: 	train_batch_size: 128


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not 

Run xrfxfl3o errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 14.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run xrfxfl3o errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 14.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: fkz6wuyj with config:
wandb: 	gradient_accumulation_steps: 16
wandb: 	learning_rate: 5.411075742634092e-05
wandb: 	max_seq_length: 128
wandb: 	train_batch_size: 32


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not 

Run fkz6wuyj errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 14.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run fkz6wuyj errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 14.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: apx2t7ha with config:
wandb: 	gradient_accumulation_steps: 8
wandb: 	learning_rate: 3.257883357078922e-05
wandb: 	max_seq_length: 256
wandb: 	train_batch_size: 32
